In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import gc
import time

In [16]:
pwd

'C:\\Users\\kjh96\\Github\\Project_Starcraft'

In [17]:
ls

 C 드라이브의 볼륨: Windows10
 볼륨 일련 번호: 96C5-6AC9

 C:\Users\kjh96\Github\Project_Starcraft 디렉터리

2020-04-13  오후 11:08    <DIR>          .
2020-04-13  오후 11:08    <DIR>          ..
2020-04-13  오후 05:56    <DIR>          .ipynb_checkpoints
2020-04-13  오후 11:08            14,820 data memory reduce.ipynb
2020-04-13  오후 06:03            15,785 data.ipynb
2020-03-31  오후 03:33       830,026,090 data.zip
2020-03-31  오후 03:57           151,099 sample_submission.csv
2020-03-31  오후 03:57     1,744,583,431 test.csv
2020-04-13  오후 06:00     2,224,215,728 test.feather
2020-03-31  오후 03:58     4,190,426,680 train.csv
2020-04-13  오후 05:59     5,732,812,224 train.feather
2020-04-13  오후 10:57                 0 train.pkl
               9개 파일      14,722,245,857 바이트
               3개 디렉터리  25,301,606,400 바이트 남음


# Load dataset

In [20]:
%%time
train = pd.read_csv('train.csv')

MemoryError: Unable to allocate 1.50 GiB for an array with shape (3, 67091776) and data type int64

In [5]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         int64  
 1   winner          int64  
 2   time            float64
 3   player          int64  
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB
None


In [6]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [7]:
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 3583.0875244140625  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  2367.3971557617188  MB
This is  66.07142972732312 % of the initial size
_________________
_________________

[]


In [8]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         float32
 1   winner          float32
 2   time            uint8  
 3   player          float32
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float32(3), object(3), uint8(1)
memory usage: 2.3+ GB
None


# Save the data with pickle

In [ ]:
train.to_csv('train.csv', index=False)

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
print("event unique values : {}".format(train.event.unique()))
print("species unique values : {}".format(train.species.unique()))

In [ ]:
event_dict = {i:j for (i,j) in zip(train.event.unique(), range(train.event.nunique()))}
print(event_dict)

In [ ]:
train['event'] = train['event'].map(event_dict)

In [ ]:
species_dict = {i:j for (i,j) in zip(train.species.unique(), range(train.species.nunique()))}
print(species_dict)

In [ ]:
train['species'] = train['species'].map(species_dict)

In [ ]:
train, NAlist = reduce_mem_usage(train)
print("-"*30)
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("-"*30)
print("")
print(NAlist)

In [ ]:
print(train.info())

In [ ]:
%%time
train.to_pickle('train.pkl')
#train.to_csv('train.csv', index=False)